## 1. Introduction
<p><a href="https://commons.wikimedia.org/wiki/File:Tux.svg">
<img align: 'right' style="float: right;margin:5px 20px 5px 1px" width="150px" src="./img/tux.png" alt="Tux - the Linux mascot"/>
</a></p>
<p>Version control repositories like CVS, Subversion or Git can be a real gold mine for software developers. They contain every change to the source code including the date (the "when"), the responsible developer (the "who"), as well as a little message that describes the intention (the "what") of a change.</p>
<p>In this notebook, we will analyze the evolution of a very famous open-source project &ndash; the Linux kernel. The Linux kernel is the heart of some Linux distributions like Debian, Ubuntu or CentOS. Our dataset at hand contains the history of kernel development of almost 13 years (early 2005 - late 2017). We get some insights into the work of the development efforts by </p>
<ul>
<li>identifying the TOP 10 contributors and</li>
<li>visualizing the commits over the years.</li>
</ul>

In [1]:
# Printing the content of git_log_excerpt.csv
with open("./datasets/git_log_excerpt.csv") as f:
    print(f.read())

1502382966#Linus Torvalds
1501368308#Max Gurtovoy
1501625560#James Smart
1501625559#James Smart
1500568442#Martin Wilck
1502273719#Xin Long
1502278684#Nikolay Borisov
1502238384#Girish Moodalbail
1502228709#Florian Fainelli
1502223836#Jon Paul Maloy


## 2. Reading in the dataset
<p>The dataset was created by using the command <code>git log --encoding=latin-1 --pretty="%at#%aN"</code> in late 2017. The <code>latin-1</code> encoded text output was saved in a header-less CSV file. In this file, each row is a commit entry with the following information:</p>
<ul>
<li><code>timestamp</code>: the time of the commit as a UNIX timestamp in seconds since 1970-01-01 00:00:00 (Git log placeholder "<code>%at</code>")</li>
<li><code>author</code>: the name of the author that performed the commit (Git log placeholder "<code>%aN</code>")</li>
</ul>
<p>The columns are separated by the number sign <code>#</code>. The complete dataset is in the <code>datasets/</code> directory. It is a <code>gz</code>-compressed csv file named <code>git_log.gz</code>.</p>

In [2]:
# Loading in the pandas module as 'pd'
import pandas as pd
import zipfile
# Reading in the log file
git_log = pd.read_csv('datasets/git_log.gz',sep ='#',header=None,  encoding ='latin-1',names=['timestamp', 'author'] )

# Printing out the first 5 rows
git_log.head()

,timestamp,author
0,1502826583,Linus Torvalds
1,1501749089,Adrian Hunter
2,1501749088,Adrian Hunter
3,1501882480,Kees Cook
4,1497271395,Rob Clark


## 3. Getting an overview
<p>The dataset contains the information about every single code contribution (a "commit") to the Linux kernel over the last 13 years. We'll first take a look at the number of authors and their commits to the repository.</p>

In [3]:
# calculating number of commits
number_of_commits = git_log.timestamp.count()

# calculating number of authors
number_of_authors = git_log.author.nunique()

# printing out the results
print("%s authors committed %s code changes." % (number_of_authors, number_of_commits))

17385 authors committed 699071 code changes.


## 4. Finding the TOP 10 contributors
<p>There are some very important people that changed the Linux kernel very often. To see if there are any bottlenecks, we take a look at the TOP 10 authors with the most commits.</p>

In [4]:
# Identifying the top 10 authors
top_10_authors = git_log.author.value_counts().nlargest(10)

# Listing contents of 'top_10_authors'
top_10_authors

Linus Torvalds           23361
David S. Miller           9106
Mark Brown                6802
Takashi Iwai              6209
Al Viro                   6006
H Hartley Sweeten         5938
Ingo Molnar               5344
Mauro Carvalho Chehab     5204
Arnd Bergmann             4890
Greg Kroah-Hartman        4580
Name: author, dtype: int64

## 5. Wrangling the data
<p>For our analysis, we want to visualize the contributions over time. For this, we use the information in the <code>timestamp</code> column to create a time series-based column.</p>

In [5]:
git_log.head()

,timestamp,author
0,1502826583,Linus Torvalds
1,1501749089,Adrian Hunter
2,1501749088,Adrian Hunter
3,1501882480,Kees Cook
4,1497271395,Rob Clark


In [6]:
# converting the timestamp column
git_log['timestamp']= pd.to_datetime(git_log['timestamp'],unit='s')

# summarizing the converted timestamp column
git_log['timestamp'].describe()

count                  699071
unique                 668448
top       2008-09-04 05:30:19
freq                       99
first     1970-01-01 00:00:01
last      2037-04-25 08:08:26
Name: timestamp, dtype: object

In [7]:
git_log.head(20)

,timestamp,author
0,2017-08-15 19:49:43,Linus Torvalds
1,2017-08-03 08:31:29,Adrian Hunter
2,2017-08-03 08:31:28,Adrian Hunter
3,2017-08-04 21:34:40,Kees Cook
4,2017-06-12 12:43:15,Rob Clark
5,2017-08-03 08:31:27,Adrian Hunter
6,2017-08-03 08:31:26,Adrian Hunter
7,2017-08-07 12:37:50,Anton Volkov
8,2017-08-01 07:11:52,Keith Busch
9,2017-08-15 14:34:55,Alex Deucher


## 6. Treating wrong timestamps
<p>As we can see from the results above, some contributors had their operating system's time incorrectly set when they committed to the repository. We'll clean up the <code>timestamp</code> column by dropping the rows with the incorrect timestamps.</p>

In [8]:
# determining the first real commit timestamp
#first_commit_timestamp = git_log.iloc[-1]['timestamp']
first_commit_timestamp = git_log[git_log['author']=='Linus Torvalds']['timestamp'].min()
print('frist_commit_timestamp',first_commit_timestamp)
# determining the last sensible commit timestamp
last_commit_timestamp = pd.to_datetime('2018')
print('last_commit_timestamp',last_commit_timestamp)
# filtering out wrong timestamps
corrected_log = git_log[
    (git_log['timestamp'] >= first_commit_timestamp) &
    (git_log['timestamp'] <= last_commit_timestamp)]

# summarizing the corrected timestamp column
print(corrected_log.describe())
print(corrected_log.head())

frist_commit_timestamp 2005-04-16 22:20:36
last_commit_timestamp 2018-01-01 00:00:00
                  timestamp          author
count                698569          698568
unique               667977           17375
top     2008-09-04 05:30:19  Linus Torvalds
freq                     99           23361
first   2005-04-16 22:20:36             NaN
last    2017-10-03 12:57:00             NaN
            timestamp          author
0 2017-08-15 19:49:43  Linus Torvalds
1 2017-08-03 08:31:29   Adrian Hunter
2 2017-08-03 08:31:28   Adrian Hunter
3 2017-08-04 21:34:40       Kees Cook
4 2017-06-12 12:43:15       Rob Clark


## 7. Grouping commits per year
<p>To find out how the development activity has increased over time, we'll group the commits by year and count them up.</p>

In [9]:
# Counting the no. commits per year
commits_per_year =corrected_log.groupby(pd.Grouper(key='timestamp', freq='AS')).count()

# Listing the first rows
commits_per_year.head()

,author
timestamp,
2005-01-01,16229
2006-01-01,29255
2007-01-01,33759
2008-01-01,48847
2009-01-01,52572


## 8. Visualizing the history of Linux
<p>Finally, we'll make a plot out of these counts to better see how the development effort on Linux has increased over the the last few years. </p>

In [ ]:
# Setting up plotting in Jupyter notebooks
%matplotlib inline

# plot the data
commits_per_year.plot(kind='bar',title="Commits per year (Linux kernel)", legend=False );

## 9.  Conclusion
<p>Thanks to the solid foundation and caretaking of Linux Torvalds, many other developers are now able to contribute to the Linux kernel as well. There is no decrease of development activity at sight!</p>